In [35]:
import pandas as pd
kaggle_input="./kaggle/input"
# kaggle_working="./kaggle/working"
kaggle_temp="./kaggle/temp1"

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)


In [36]:

train_data=pd.read_csv(f"{kaggle_input}/train.csv")
test_data=pd.read_csv(f"{kaggle_input}/test.csv")
# test_data[:1]
# train_data[:50]


format_test_data=True
f_sample=len(train_data)
f_data_name='f_train.csv'
if format_test_data:
    train_data=test_data
    f_sample=len(train_data)
    f_data_name='f_test.csv'



In [37]:
for i in range(f_sample):
    # eeg_id=train_data.loc[i,'eeg_id']
    # print(eeg_id)
    # eeg_data=pd.read_parquet(f"{kaggle_input}/train_eegs/{eeg_id}.parquet")
    # print(eeg_data)
    spect_id=train_data.loc[i,'spectrogram_id']
    # print(spect_id)
    spect_data=pd.read_parquet(f"{kaggle_input}/{'train' if format_test_data==False else 'test'}_spectrograms/{spect_id}.parquet")
    # print(spect_data)
    spect_col='LL_0.98'
    avgs=[sum(spect_data.loc[i:i+5,spect_col])/5 for i in range(0,min(len(spect_data),300),5)]
    # print(len(avgs))
    # print(avgs)
    # print("encoding vals in train_data")
    for t in range(len(avgs)):
        # if f'savg{t}' not in train_data.columns:
        #     train_data.insert(loc = 9,column = f'savg{t}', value = 0)
        train_data.loc[i,f'savg{t}']=avgs[t]

train_data



,spectrogram_id,eeg_id,patient_id,savg0,savg1,savg2,savg3,savg4,savg5,savg6,savg7,savg8,savg9,savg10,savg11,savg12,savg13,savg14,savg15,savg16,savg17,savg18,savg19,savg20,savg21,savg22,savg23,savg24,savg25,savg26,savg27,savg28,savg29,savg30,savg31,savg32,savg33,savg34,savg35,savg36,savg37,savg38,savg39,savg40,savg41,savg42,savg43,savg44,savg45,savg46,savg47,savg48,savg49,savg50,savg51,savg52,savg53,savg54,savg55,savg56,savg57,savg58,savg59
0,853520,3911565283,6885,17.354,11.98,10.194,16.704,17.682,23.534,24.018,22.434,20.336,18.93,18.946,29.3,28.138,34.234,18.818,18.772,22.77,17.428,25.982001,17.426,26.538,25.642,23.042,41.167999,16.536,49.926,60.354,14.058,26.298,30.496,14.34,19.976,24.206,15.472,15.792,19.744,21.91,15.718,11.544,15.816,30.906,23.314,15.174,16.274,21.142,17.216,18.132,33.684,21.231999,19.258,20.816,16.148,11.624,33.578,30.16,19.876,13.98,22.292,11.792,13.432


In [38]:

vote_cols=['seizure_vote','lpd_vote','gpd_vote','lrda_vote','grda_vote','other_vote']
savg_cols=[f'savg{i}' for i in range(60)]
train_data=train_data[[col for col in vote_cols+savg_cols] if not format_test_data else [col for col in ['eeg_id']+savg_cols]]
if format_test_data==False:
    # converting votes to probs
    for i in range(f_sample):
        sum_votes=sum(train_data.loc[i,vote_cols])
        train_data.loc[i,vote_cols]=train_data.loc[i,vote_cols].apply(lambda x: (x/sum_votes) )

train_data


,eeg_id,savg0,savg1,savg2,savg3,savg4,savg5,savg6,savg7,savg8,savg9,savg10,savg11,savg12,savg13,savg14,savg15,savg16,savg17,savg18,savg19,savg20,savg21,savg22,savg23,savg24,savg25,savg26,savg27,savg28,savg29,savg30,savg31,savg32,savg33,savg34,savg35,savg36,savg37,savg38,savg39,savg40,savg41,savg42,savg43,savg44,savg45,savg46,savg47,savg48,savg49,savg50,savg51,savg52,savg53,savg54,savg55,savg56,savg57,savg58,savg59
0,3911565283,17.354,11.98,10.194,16.704,17.682,23.534,24.018,22.434,20.336,18.93,18.946,29.3,28.138,34.234,18.818,18.772,22.77,17.428,25.982001,17.426,26.538,25.642,23.042,41.167999,16.536,49.926,60.354,14.058,26.298,30.496,14.34,19.976,24.206,15.472,15.792,19.744,21.91,15.718,11.544,15.816,30.906,23.314,15.174,16.274,21.142,17.216,18.132,33.684,21.231999,19.258,20.816,16.148,11.624,33.578,30.16,19.876,13.98,22.292,11.792,13.432


In [39]:
train_data.to_csv(f'{kaggle_temp}/{f_data_name}',index=False)